In [3]:
import numpy as np 
import matplotlib.pyplot as plt
%pylab inline

#Link to my google drive
from google.colab import drive
drive.mount('/content/gdrive')

Populating the interactive namespace from numpy and matplotlib
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Load images with numpy
images_train = np.load('/content/gdrive/My Drive/IFT3395-TP4/train_images.npy', encoding='latin1')
images_train = images_train[:,1]/255

#Load labels
train_labels = np.genfromtxt('/content/gdrive/My Drive/IFT3395-TP4/train_labels.csv', names=True, delimiter=',', dtype=[('Id', 'i8'), ('Category', 'U15')])


In [0]:
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

labEncoder = LabelEncoder()
labEncoder.fit(train_labels['Category'])
label = labEncoder.transform(train_labels['Category'])

# Transformer les catégories en codage OneHot
# oneHot = OneHotEncoder(n_values=num_classes, sparse=False)
# labelOneHot = oneHot.fit_transform(label.reshape(-1,1))
# oneHot.fit(label.reshape(-1,1))
# labelOneHot = oneHot.transform(label.reshape(-1,1))

In [0]:
#Preprocessing images by reducing noise

class block:
  
    def __init__(self, d, x, y):
      queue = [(x,y)]
      self.visited = []
      est = x
      west = x
      north = y
      south = y
      flag = np.zeros((d.shape[0],d.shape[1]))

      while queue:
        
#           print("queue is", queue)
          s = queue.pop(0)
          self.visited.append(s)
          flag[s[0]][s[1]] = 1
        
#         if not flag[s[0]][s[1]]:
          

          if s[0]+1 < 100 and d[s[0]+1][s[1]] > 0 and not flag[s[0]+1][s[1]]:
            queue.append((s[0]+1,s[1]))
            flag[s[0]+1][s[1]] = 1
            if s[0]+1 > west:
              west = s[0]+1

          if s[1]+1 < 100 and d[s[0]][s[1]+1] > 0 and not flag[s[0]][s[1]+1]:
            queue.append((s[0],s[1]+1))
            flag[s[0]][s[1]+1] = 1
            if s[1]+1 > south:
              south = s[1]+1

          if s[0]-1 > 0 and d[s[0]-1][s[1]] > 0 and not flag[s[0]-1][s[1]]:
            queue.append((s[0]-1,s[1]))
            flag[s[0]-1][s[1]] = 1
            if s[0]-1 < est:
              est = s[0]-1

          if s[1]-1 > 0 and d[s[0]][s[1]-1] > 0 and not flag[s[0]][s[1]-1]:
            queue.append((s[0],s[1]-1))
            flag[s[0]][s[1]-1] = 1
            if s[1]-1 < north:
              north = s[1]-1

          if s[0]+1 < 100 and s[1]+1 < 100 and d[s[0]+1][s[1]+1] > 0 and not flag[s[0]+1][s[1]+1]:
            queue.append((s[0]+1,s[1]+1))
            flag[s[0]+1][s[1]+1] = 1
            if s[0]+1 > west:
              west = s[0]+1
            if s[1]+1 > south:
              south = s[1]+1

          if s[0]-1 > 0 and s[1]-1 > 0 and d[s[0]-1][s[1]-1] > 0 and not flag[s[0]-1][s[1]-1]:
            queue.append((s[0]-1,s[1]-1))
            flag[s[0]-1][s[1]-1] = 1
            if s[0]-1 < est:
              est = s[0]-1
            if s[1]-1 < north:
              north = s[1]-1

          if s[0]+1 < 100 and s[1]-1 > 0 and d[s[0]+1][s[1]-1] > 0 and not flag[s[0]+1][s[1]-1]:
            queue.append((s[0]+1,s[1]-1))
            flag[s[0]+1][s[1]-1] = 1
            if s[0]+1 > west:
              west = s[0]+1
            if s[1]-1 < north:
              north = s[1]-1

          if s[0]-1 > 0 and s[1]+1 < 100 and d[s[0]-1][s[1]+1] > 0 and not flag[s[0]-1][s[1]+1]:
            queue.append((s[0]-1,s[1]+1))
            flag[s[0]-1][s[1]+1] = 1
            if s[0]-1 < est:
              est = s[0]-1
            if s[1]+1 > south:
              south = s[1]+1

      self.size = (west-est+1, south-north+1)

def clean_noise(d, noise_size):
    negative = []
    for i in range(d.shape[0]):
      for j in range(d.shape[1]):
        if d[i][j] > 0:
          b = block(d,i,j)
          if b.size < (noise_size[0],noise_size[1]):
            for dots in b.visited:
              d[dots[0]][dots[1]] = 0 
          else:
            negative.append(b.visited)
            for dots in b.visited:
              d[dots[0]][dots[1]] = -d[dots[0]][dots[1]]
    
    biggest = 0
    for h in negative:
      if len(h) > biggest:
        biggest = len(h)
    
    for g in negative:
        if len(g) == biggest:
          for p in g:
            d[p[0]][p[1]] = -d[p[0]][p[1]]
        else:
          for p in g:
            d[p[0]][p[1]] = 0
    
    return d
  
  
def centralize(image, dimInitial, dimCentre):
#Find bounding box of the image
  col_sum = np.where(np.sum(image, axis = 0)>0)
  row_sum = np.where(np.sum(image, axis = 1)>0)
  
  if len(col_sum[0])== 0 and len(row_sum[0]) == 0 :
    return image[0:dimCentre, 0:dimCentre]
  
  y1 = row_sum[0][0]
  x1 = col_sum[0][0]
  
  padX, padY = 0, 0
  y2 = y1 + dimCentre
  x2 = x1 + dimCentre
  
  if x2 > dimInitial:
    padX = x2 - dimInitial
    x2 = dimInitial
    
  if y2 > dimInitial:
    padY = y2 - dimInitial
    y2 = dimInitial
    
  img = image[y1:y2, x1:x2]
  
  if padX > 0:
    pad = np.zeros((y2-y1, padX))
    img = np.concatenate((img, pad), axis = 1)
  
  if padY > 0:
    pad = np.zeros((padY, dimCentre))
    img = np.concatenate((img, pad), axis = 0)
    
  return img


# image = images_train[44].reshape(100,100)
# print("Before noise reduction")
# plt.imshow(image)
# plt.show()
# # for i in range(image.shape[0]):
# #   for j in range(image.shape[1]):
# #     if image[i][j]<0:
# #       print(image[i][j])

# clean_noise(image,noise_size=(20,20))

# print("After noise reduction")
# plt.imshow(image)
# plt.show()

In [13]:
# # chargment de données
# trainSet = np.zeros((num_train, num_features))
# for i in range(num_train):
#     trainSet[i] = images_train[i].tolist()

from skimage.transform import rescale
num_train = images_train.shape[0]
num_features = 32*32
num_classes = 31

#Nettoyer les bruits
trainProcessed = np.zeros((num_train, num_features))
for i in range(num_train):
  image = images_train[i].reshape(100,100)
  clean_noise(image.reshape(100,100),noise_size=(12,12))
  image = centralize(image,100,32)  
  trainProcessed[i] = image.reshape(1, 32*32)
  
# for i in range(10):
#   plt.imshow(trainProcessed[i].reshape(50,50))
#   plt.show()
 
trainProcessed

# for i in range(10):
#   plt.imshow(images_train[i].reshape(100,100))
#   plt.show()
  

# trainProcessed = trainProcessed.reshape(-1,1,100,100)

#Shuffle les donnees
# randomIdx = np.arange(num_train)
# np.random.seed(50)
# np.random.shuffle(randomIdx)
# trainSetRand = trainSet[randomIdx]
# labelRand = label[randomIdx]

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.99607843, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [14]:
trainBatches = trainProcessed[:9000]
labelTrain = label[:9000]
validBatches = trainProcessed[9000:]
labelValid = label[9000:]
print(trainBatches.shape, labelTrain.shape)
print(validBatches.shape, labelValid.shape)

(9000, 1024) (9000,)
(1000, 1024) (1000,)


In [16]:
from sklearn.svm import SVC
from sklearn import metrics
import time

for c in [0.0005,0.001,0.005,.01]:
    start = time.time()
    svc = SVC(kernel='linear', C=c)
    svc.fit(trainBatches, labelTrain)
    predictions = svc.predict(validBatches)
    accuracy = metrics.accuracy_score(predictions, labelValid)
    end = time.time()
    duree = end - start
    
    print('c =',c, 'precision =',accuracy, 'durée =',duree)

c = 0.0005 precision = 0.264 durée = 156.77781414985657
c = 0.001 precision = 0.319 durée = 140.4801890850067
c = 0.005 precision = 0.415 durée = 107.04175639152527
c = 0.01 precision = 0.398 durée = 100.05288243293762
